# Retrieval Augmented Generation with LangChain

In [ ]:
import sys
import os

# Use current working directory and go one level up
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

# Now you can import your config
from config import api_key

from openai import OpenAI

## Chapter 1 - Building RAG applications with LangChain

### Section 1.1 - Loading documents for RAG with LangChain

#### Loading PDF files for RAG
To begin implementing Retrieval Augmented Generation (RAG), you'll first need to load the documents that the model will access. These documents can come from a variety of sources, and LangChain supports document loaders for many of them.

In this exercise, you'll use a document loader to load a PDF document containing the paper, Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks by Lewis et al. (2021). This file is available for you as `'rag_paper.pdf'`.

Note: `pypdf`, a dependency for loading PDF documents in LangChain, has already been installed for you.

In [ ]:
# Import library
from langchain_community.document_loaders import PyPDFLoader

# Create a document loader for rag_paper.pdf
loader = PyPDFLoader('./data/rag-paper.pdf')

# Load the document
data = loader.load()
print(data[0])

#### Loading HTML files for RAG

It's possible to load documents from many different formats, including complex formats like HTML.

If you're not familiar with HTML, it's a markup language for creating web pages. Here's a small example:

In this exercise, you'll load an HTML file taken containing a DataCamp blog post webpage. The necessary classes have already been imported for you.

In [ ]:
# Import library
from langchain_community.document_loaders import UnstructuredHTMLLoader

# Create a document loader for unstructured HTML
loader = UnstructuredHTMLLoader('./data/datacamp-blog.html')

# Print the first document's content
print(data[0].page_content)

# Print the first document's metadata
print(data[0].metadata)

### Section 1.2 - Text splitting, embedding and vector storage

In [ ]:
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

#### Getting started with text splitting

Time to start splitting! You've been provided with a statement about RAG stored in the string variable `text`. Your job is to split this string on occurrences of the `'.'` character. Take a look at the splitting results to see how this strategy performed.

In [ ]:
text = '''RAG (retrieval augmented generation) is an advanced NLP model that combines retrieval mechanisms with generative capabilities. RAG aims to improve the accuracy and relevance of its outputs by grounding responses in precise, contextually appropriate data.'''

# Define a text splitter that splits on the '.' character
text_splitter = CharacterTextSplitter(
    separator=".",
    chunk_size=75,  
    chunk_overlap=10  
)

# Split the text using text_splitter
chunks = text_splitter.split_text(text)
print(chunks)
print([len(chunk) for chunk in chunks])

#### Recursively splitting documents

Splitting on a single character is simple and predictable, but it often produces sub-optimal chunks. In this exercise, you'll apply recursive character splitting to split the Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks paper you loaded in a earlier exercise.

Recall that recursive character splitting iterates over a list of characters, splitting on each in turn to see if chunks can be created beneath the `chunk_size` limit.


In [ ]:
loader = PyPDFLoader("./data/rag-paper.pdf")
document = loader.load()

# Define a text splitter that splits recursively through the character list
text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n', '.', ' ', ''],
    chunk_size=1000,  
    chunk_overlap=100  
)

# Split the document using text_splitter
chunks = text_splitter.split_documents(document)
print(chunks)
print([len(chunk.page_content) for chunk in chunks])
print(len(chunks))

In [ ]:
# vector_store.search("What is BART?", search_type="similarity")

In [ ]:
vector_store.get(limit=1)['documents']

#### Embedding and storing documents
The final step for preparing the documents for retrieval is embedding and storing them. You'll be using the text-embedding-3-small model from OpenAI for embedding the chunked documents, and storing them in a local Chroma vector database.

The `chunks` you created from splitting the Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks paper recursively have been pre-loaded.

Creating and using an OpenAI API key is not required in this exercise. You can leave the `<OPENAI_API_TOKEN>` placeholder, which will send valid requests to the OpenAI API.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# Initialize the OpenAI embedding model
embedding_model = OpenAIEmbeddings(
    api_key=api_key, 
    model='text-embedding-3-small')

# Create a Chroma vector store and embed the chunks
vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embedding_model,
    persist_directory="./chromadb/"
)

### Section 1.3 - Building an LCEL retrieval chain

#### Creating the retrieval prompt

A key piece of any RAG implementation is the retrieval prompt. In this exercise, you'll create a chat prompt template for your retrieval chain and test that the LLM is able to respond using only the context provided.

An `llm` has already been defined for you to use.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)

In [ ]:
prompt = """
Use the only the context provided to answer the following question. If you don't know the answer, reply that you are unsure.
Context: {context}
Question: {question}
"""

# Convert the string into a chat prompt template
prompt_template = ChatPromptTemplate.from_template(prompt)

# Create an LCEL chain to test the prompt
chain = prompt_template | llm

# Invoke the chain on the inputs provided
print(chain.invoke({"context": "DataCamp's RAG course was created by Meri Nova and James Chapman!", "question": "Who created DataCamp's RAG course?"}))

#### Building the retrieval chain
Now for the finale of the chapter! You'll create a retrieval chain using LangChain's Expression Language (LCEL). This will combine the vector store containing your embedded document chunks from the RAG paper you loaded earlier, a prompt template, and an LLM so you can begin talking to your documents.

Here's a reminder of the prompt_template you created in the previous exercise, and which is available for you to use:

The vector_store of embedded document chunks that you created previously has also been loaded for you, along with all of the libraries and classes required.

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Convert the vector store into a retriever
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":2})

# Create the LCEL retrieval chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

# Invoke the chain
#print(chain.invoke("Who are the authors?"))
#print(chain.invoke("What is BART?"))
print(chain.invoke("What is the broader impact"))

## Chapter 2 - Improving the RAG Architecture

In [ ]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

#### Loading code files
Chatbots can not only access text files, but also code files like Python `(.py)` and Markdown files `(.md)`. In this exercise, you'll load a Python file containing the RAG architecture you created in Chapter 1. Let's load the file to get a reminder!

All of the classes needed to complete this exercise are already loaded.

In [ ]:
# Create a document loader for README.md and load it
loader = UnstructuredMarkdownLoader('./data/README.md')

markdown_data = loader.load()
print(markdown_data[0])

In [ ]:
from langchain_community.document_loaders import PythonLoader

In [ ]:
# Create a document loader for rag.py and load it
loader = PythonLoader('rag.py')

python_data = loader.load()
print(python_data[0])

#### Splitting Python files

Although text and code files contain the same characters, code files contain structures beyond natural language. To retain this code-specific context during document splitting, you should program the splitter to first try to split on the most common code structure. Fortunately, LangChain provides functionality to do just that!

All of the necessary classes have been imported for you, including `Language` from `langchain_text_splitters`.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter, Language

# Create a Python-aware recursive character splitter
python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=300, chunk_overlap=100
)

# Split the Python content into chunks
chunks = python_splitter.split_documents(python_data)

for i, chunk in enumerate(chunks[:3]):
    print(f"Chunk {i+1}:\n{chunk.page_content}\n")

### Section 2.2 - Advanced splitting method

#### Splitting by tokens

Splitting documents using RecursiveCharacterTextSplitter or CharacterTextSplitter is convenient, and can give you good performance in some cases, but it does have one drawback: they split using characters as base units, rather than tokens, which are processed by the model.

In this exercise, you'll split documents using a token text splitter, so you can verify the number of tokens in each chunk to ensure that they don't exceed the model's context window. A PDF document has been loaded as `document`.

`tiktoken` and all necessary classes have been imported for you.

In [ ]:
import tiktoken
from langchain_text_splitters import TokenTextSplitter
from langchain_community.document_loaders import PyPDFLoader

# Create a document loader for rag_paper.pdf
loader = PyPDFLoader('./data/rag-paper.pdf')

# Load the document
data = loader.load()
#print(data[0])

# Get the encoding for gpt-4o-mini
encoding = tiktoken.encoding_for_model('gpt-4o-mini')

# Create a token text splitter
token_splitter = TokenTextSplitter(encoding_name=encoding.name, chunk_size=100, chunk_overlap=10)

# Split the PDF into chunks
chunks = token_splitter.split_documents(document)

for i, chunk in enumerate(chunks[:3]):
    print(f"Chunk {i+1}:\nNo. tokens: {len(encoding.encode(chunk.page_content))}\n{chunk}\n")

#### Splitting semantically

All of the splitting strategies you've used up to this point have the same drawback: the split doesn't consider the context of the surrounding text, so context can easily be lost during splitting.

In this exercise, you'll create and apply a semantic text splitter, which is a cutting-edge experimental method for splitting text based on semantic meaning. When the splitter detects that the meaning of the text has deviated past a certain threshold, a split will be performed.

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.document_loaders import PyPDFLoader

# Create a document loader for rag_paper.pdf
loader = PyPDFLoader('./data/rag-paper.pdf')
# Instantiate an OpenAI embeddings model
embedding_model = OpenAIEmbeddings(api_key=api_key, model='text-embedding-3-small')

# Create the semantic text splitter with desired parameters
semantic_splitter = SemanticChunker(
    embeddings=embedding_model, breakpoint_threshold_type="gradient", breakpoint_threshold_amount=0.8
)

# Split the document
chunks = semantic_splitter.split_documents(document)
print(chunks[0])

### Section 2.3 - Optimizing document retrieval

In [ ]:
from langchain_community.retrievers import BM25Retriever

#### Understanding BM25

Before you start integrating a BM25 sparse retriever into your RAG architecture, it's best to test it on some short strings to get a intuition for how the retriever selects the documents.

You've been provided with three strings that you'll use as the basis for your BM25 retriever. The functionality required for this exercise is already loaded for you.

In [ ]:
chunks = [
    "RAG stands for Retrieval Augmented Generation.",
    "Graph Retrieval Augmented Generation uses graphs to store and utilize relationships between documents in the retrieval process.",
    "There are different types of RAG architectures; for example, Graph RAG."
]

# Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_texts(chunks, k=3)

# Invoke the retriever
results = bm25_retriever.invoke("Graph RAG")

# Extract the page content from the first result
print("Most Relevant Document:")
print(results[0].page_content)

#### Sparse retrieval with BM25

Time to try out a sparse retrieval implementation! You'll create a BM25 retriever to ask questions about an academic paper on RAG, which has already been split into chunks called chunks. An OpenAI chat model and prompt have also been defined as `llm` and `prompt`, respectively. You can view the prompt provided by printing it in the console.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)

In [ ]:
# Create a document loader for rag_paper.pdf
loader = PyPDFLoader('./data/rag-paper.pdf')
# Instantiate an OpenAI embeddings model
embedding_model = OpenAIEmbeddings(api_key=api_key, model='text-embedding-3-small')

# Create the semantic text splitter with desired parameters
semantic_splitter = SemanticChunker(
    embeddings=embedding_model, breakpoint_threshold_type="gradient", breakpoint_threshold_amount=0.8
)

# Split the document
chunks = semantic_splitter.split_documents(document)
# print(chunks[0])

In [ ]:
prompt_string  ="""
Use the only the context provided to answer the following question. If you don't know the answer, reply that you are unsure.
Context: {context}
Question: {question}\n"""

prompt = ChatPromptTemplate.from_template(prompt_string)

In [ ]:
# Create a BM25 retriever from chunks
retriever = BM25Retriever.from_documents(
    documents=chunks, 
    k=5
)

# Create the LCEL retrieval chain
chain = ({"context": retriever, "question": RunnablePassthrough()}
         | prompt
         | llm
         | StrOutputParser()
)

# Invoke the chain
print(chain.invoke("What are knowledge-intensive tasks?"))

### Section 2.4 - Introduction to RAG evaluation

**remark** - I found the video unclear. The matter is rather complex is very briefly introduced and too difficult to capture from the video. 

#### Ragas context precision evaluation
To start your RAG evaluation journey, you'll begin by evaluating the context precision RAG metric using the `ragas` framework. Recall that context precision is essentially a measure of how relevant the retrieved documents are to the input query.

In this exercise, you've been provided with an input query, and the documents retrieved by a RAG application, and the ground truth, which was the most appropriate document to retrieve based on the opinion of a human expert. You'll calculate the context precision on these strings before evaluating an actual LangChain RAG chain in the next exercise.

The text generated by the RAG application has been saved to the variable `model_response` for brevity.

In [ ]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key=api_key)

In [ ]:
from ragas.metrics import context_precision
from ragas.integrations.langchain import EvaluatorChain

# Define the context precision chain
context_precision_chain = EvaluatorChain(metric=context_precision, llm=llm, embeddings=embeddings)

# Evaluate the context precision of the RAG chain
eval_result = context_precision_chain({
  "question": "How does RAG enable AI applications?",
  "ground_truth": "RAG enables AI applications by integrating external data in generative models.",
  "contexts": [
    "RAG enables AI applications by integrating external data in generative models.",
    "RAG enables AI applications such as semantic search engines, recommendation systems, and context-aware chatbots."
  ]
})

print(f"Context Precision: {eval_result['context_precision']}")

#### Ragas faithfulness evaluation

In this exercise, you'll evaluate the faithfulness of the RAG architecture you created at the end of Chapter 1. This chain has been re-defined for you and is available as through the variable `chain`.

You'll use the query provided, the chain's output, and the retrieved output to evaluate the faithfulness using the `ragas` framework.

The classes required have already been imported for you.

#### String evaluation
Time to really evaluate the final output by comparing it to an answer written by a subject matter expert. You'll use LangSmith's `LangChainStringEvaluator` class to perform this string comparison evaluation.

A `prompt_template` for string evaluation has already been written for you as:

The output from the RAG chain is stored as `predicted_answer` and the expert's response is stored as `ref_answer`.

All of the necessary classes have been imported for you.

**remark** - below we need to do quite some setup for the exercise

In [ ]:
from langchain_openai import ChatOpenAI
from langsmith.evaluation import LangChainStringEvaluator
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

eval_llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key, temperature=0)

prompt = """You are an expert professor specialized in grading students' answers to questions.
You are grading the following question:{query}
Here is the real answer:{answer}
You are grading the following predicted answer:{result}
Respond with CORRECT or INCORRECT:
Grade:"""

prompt_template = PromptTemplate(
    input_variables=["query", "answer", "result"],
    template= prompt)

prompt_template

query = "How does RAG improve question answering with LLMs"
predicted_answer = "RAG improves question answering with LLMs by generating correct answers even when the correct answer is not present in any retrieved document, achieving a notable accuracy of 11.8% in such cases, while extractive models would score 0%. Additionally, RAG models outperform other models like BART in terms of generating factually correct and diverse text, as well as being able to answer questions in a more flexible, abstractive manner rather than relying solely on extractive methods."
ref_answer = "Retrieval-Augmented Generation (RAG) improves question answering with large language models (LLMs) by combining a retrieval mechanism with a generative model. The retrieval system fetches relevant documents or passages from external knowledge sources, giving the LLM access to more up-to-date and accurate information than what it has learned during training. This allows RAG to generate responses that are grounded in factual data, reducing the risk of hallucination and improving the model's accuracy, especially in niche or specialized domains where the LLM alone may lack expertise. By leveraging both external knowledge and the generative abilities of LLMs, RAG enhances the quality, relevance, and factuality of the answers provided."

**exercise**

In [ ]:
# Create the QA string evaluator
qa_evaluator = LangChainStringEvaluator(
    "qa",
    config={
        "llm": eval_llm,
        "prompt": prompt_template
    }
)

query = "How does RAG improve question answering with LLMs?"

# Evaluate the RAG output by evaluating strings
score = qa_evaluator.evaluator.evaluate_strings(
    prediction=predicted_answer,
    reference=ref_answer,
    input=query
)

print(f"Score: {score}")

oh dear! Looks like this RAG application needs to go back to the drawing board. Perhaps some of the techniques learned in this chapter, like semantic splitting or sparse retrieval, would improve this metric, or perhaps tweaking the retrieval prompt to allow a bit more creativity.

## Chapter 3 - Introduction to Graph RAG

### Section 3.1 - From vectors to graphs

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# Original document
doc = Document(page_content="""
The 20th century witnessed the rise of some of the most influential scientists in history, 
with Albert Einstein and Marie Curie standing out among them. Einstein, best known for his theory of relativity, 
revolutionized our understanding of space, time, and energy, earning him the Nobel Prize in Physics in 1921 
for his explanation of the photoelectric effect. Marie Curie, a pioneer in the study of radioactivity, 
was the first woman to win a Nobel Prize. She was awarded the Nobel Prize in Physics in 1903, 
shared with her husband Pierre Curie and Henri Becquerel, for their work on radiation. 
Curie later made history again by winning a second Nobel Prize in Chemistry in 1911 
for her discoveries of radium and polonium. Both scientists made monumental contributions 
that continue to influence the fields of physics and beyond.
""")

# Splitter configuration
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=50,
)

# Split into smaller docs
docs = splitter.split_documents([doc])

In [ ]:
docs

In [ ]:
# Define the LLM
llm = ChatOpenAI(api_key=api_key, model="gpt-4o-mini", temperature=0)

# Instantiate the LLM graph transformer
llm_transformer = LLMGraphTransformer(llm=llm)

# Convert the text documents to graph documents
graph_documents = llm_transformer.convert_to_graph_documents(docs)
print(f"Derived Nodes:\n{graph_documents[0].nodes}\n")
print(f"Derived Edges:\n{graph_documents[0].relationships}")

#### Getting to know graphs

Graphs of nodes and edges are to Graph RAG what vectors and vector distances are to vector RAG. Graphs, however, are much better able to capture the relationship between different entities in the text, rather than relying on semantic meaning to return of the relevant context.

Consider the small graph that closely resembles that one you created in the previous exercise.

Which of the following statements about the graph shown is correct?

### Section 3.2 - Storing and quering documents

**Intermezzo - setting up neo4j using docker**

visit docker hub for more details: https://hub.docker.com/_/neo4j

This runs Neo4j in a container with:

- HTTP interface at http://localhost:7474
- Bolt protocol on port 7687 (used by Python client)
- Default username: neo4j
- Password: ThisIsAStrongPassword123!

please note that I did not include persistent storage.

In [ ]:
from neo4j import GraphDatabase

# Connection URI and credentials
uri = "bolt://localhost:7687"
user = "neo4j"
password = "ThisIsAStrongPassword123!"

# Create driver
driver = GraphDatabase.driver(uri, auth=(user, password))

# Example query
def create_and_query(tx):
    # Create a node
    tx.run("CREATE (:Person {name: $name})", name="Ada Lovelace")
    # Return all people
    result = tx.run("MATCH (p:Person) RETURN p.name AS name")
    return [record["name"] for record in result]

# Run the query
with driver.session() as session:
    names = session.execute_write(create_and_query)
    print("People in the database:", names)

# Close the driver
driver.close()

**Neo4j connect langchain**

In [ ]:
from langchain_community.graphs import Neo4jGraph
#from langchain_neo4j import Neo4jGraph

In [ ]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="ThisIsAStrongPassword123!")

#### Building-up your graph database
So that you don't have to regenerate your graph documents every time, it's best practice to store them in a database that's specifically designed for graph data. Neo4j graph databases are an excellent choice for graph storage and retrieval, so you'll set one up using LangChain's Neo4j functionality.

Note: to use Neo4j in LangChain, you must also have the `neo4j` library installed as a dependency. In this course, this has already been done for you.

In [ ]:
# Add the graph documents, sources, and include entity labels
graph.add_graph_documents(
    graph_documents,
    include_source=True,
    baseEntityLabel=True
)

graph.refresh_schema()

# Print the graph schema
print(graph.get_schema)

#### Querying your graph database
Now that you have your database set up, it's time to begin querying. You'll view the graph schema to refamiliarize yourself with the nodes and relationships, and then write a Cypher query to query the graph.

If you need help with writing or fixing your Cypher query, you can use the `ask_chatgpt(text: str)` function that we've defined for you, which accepts a string argument.

The `graph` you created previously is available for you to use.

In [ ]:
# Print the graph schema
print(graph.get_schema)

# Query the graph
results = graph.query("""
MATCH (relativity:Concept {id: "Theory Of Relativity"}) <-[:KNOWN_FOR]- (scientist)
RETURN scientist
""")

print(results[0])

### Section 3.3 - Creating the Graph RAG chain

#### Chaining, Graph RAG style!
Now to bring everything together to create a Graph RAG QA chain! You've been provided with the same `graph` you've worked with throughout this chapter (with some potential variation in the specific nodes and relationships), and you'll connect this with another LLM to generate the Cypher query and return the natural language response.

In [ ]:
print(graph.get_schema)

### Section 3.4 - improving graph retrieval

In [ ]:
from langchain_community.graphs.neo4j_graph import Neo4jGraph
from langchain_community.chains.graph_qa.cypher import GraphCypherQAChain
from langchain_openai import ChatOpenAI

# # Initialize the Neo4jGraph
# graph = Neo4jGraph(
#     url="bolt://localhost:7687",
#     username="neo4j",
#     password="ThisIsAStrongPassword123!"
# )

# Create the Graph Cypher QA chain
graph_qa_chain = GraphCypherQAChain.from_llm(
    llm=ChatOpenAI(api_key=api_key, temperature=0), 
    graph=graph, 
    allow_dangerous_requests=True,
    validate_cypher=True,
    verbose=True
)

# Invoke the chain with the input provided
result = graph_qa_chain.invoke({"query": "Who discovered the element Radium?"})

# Print the result text
print(f"Final answer: {result['result']}")

#### Graph RAG with filtering

For large and complex graphs, LLMs can sometimes struggle to accurately infer the most relevant nodes and relationships to build the Cypher query. Quite often, you will only need the LLM to be aware of a subset of the graph, and excluding particular node types will not only make it easier for the LLM to accurately create the Cypher query, but it will improve the query latency.

The graph database you've been working with is available as `graph`.

In [ ]:
# Create the graph QA chain excluding Concept
graph_qa_chain = GraphCypherQAChain.from_llm(
    llm=llm, 
    graph=graph, 
    allow_dangerous_requests=True,
    verbose=True, 
    exclude_types=["Concept"]
)

# Invoke the chain with the input provided
result = graph_qa_chain.invoke({"query": "Who was Marie Curie married to?"})
print(f"Final answer: {result['result']}")

#### Validating Cypher queries

When the LLMs generate the Cypher query, they have the graph schema available for reference; however, this doesn't mean there's absolute certainty that the query will reflect the schema perfectly. To improve reliability, you can validate and fix the generated query against the schema, which is particularly well-suited to fixing incorrect relationship directions.

In [ ]:
# Create the graph QA chain, validating the generated Cypher query
graph_qa_chain = GraphCypherQAChain.from_llm(
    llm=llm, 
    graph=graph, 
    verbose=True,
    validate_cypher=True,
    allow_dangerous_requests=True
)


# Invoke the chain with the input provided
result = graph_qa_chain.invoke({"query": "Who won the Nobel Prize In Physics?"})
print(f"Final answer: {result['result']}")

#### Creating a Cypher few-shot prompt

The final technique you'll utilize to improve the reliability of the generated Cypher is providing a few-shot prompt. Few-shot prompts are a great way of steering a model toward a desired output without needing to fine-tune it on a large dataset of examples.

A set of `examples` tailored to this particular use case is available as examples; feel free to print it in the shell to view its contents. You'll use these to create a few-shot prompt for the Cypher generation process. The graph you created before is still available as `graph`.

In [ ]:
from langchain_core.prompts import FewShotPromptTemplate

examples = [{'question': 'How many scientists are mentioned in the graph?',
  'query': 'MATCH (p:Person) RETURN count(DISTINCT p)'}]

# Create an example prompt template
example_prompt = PromptTemplate.from_template(
    "User input: {question}\nCypher query: {query}"
)

# Create the few-shot prompt template
cypher_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="You are a Neo4j expert. Given an input question, create a syntactically correct Cypher query to run.\n\nHere is the schema information\n{schema}.\n\nBelow are a number of examples of questions and their corresponding Cypher queries.",
    suffix="User input: {question}\nCypher query: ",
    input_variables=["question"]
)

# Create the graph Cypher QA chain
graph_qa_chain = GraphCypherQAChain.from_llm(
    graph=graph, 
    llm=llm, 
    cypher_prompt=cypher_prompt,
    verbose=True, 
    validate_cypher=True,
    allow_dangerous_requests=True
)

# Invoke the chain with the input provided
result = graph_qa_chain.invoke({"query": "Which scientist proposed the Theory Of Relativity?"})
print(f"Final answer: {result['result']}")